In [4]:
import sparknlp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pyspark.sql import SparkSession
from pyspark.sql import Row
# from pyspark.sql.types import StringType, StructType, StructField
from pyspark.sql.types import *
# import pyspark.sql.functions as F
from pyspark.sql.functions import *

import warnings
warnings.filterwarnings('ignore')

spark = sparknlp.start()

:: loading settings :: url = jar:file:/opt/module/spark-3.5.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a405a032-7eb7-4258-809a-62aa132ea318;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 in central
:: resolution report :: resolve 117ms :: artifacts dl 12ms
	:: modules in use:
	com.johnsnowlabs.nlp#spark-nlp_2.12;5.3.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-a405a032-7eb7-4258-809a-62aa132

In [5]:
# spark = SparkSession.builder \
#     .appName("Spark NLP") \
#     .master("local[*]") \
#     .config("spark.driver.memory", "16G") \
#     .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
#     .config("spark.kryoserializer.buffer.max", "2000M") \
#     .config("spark.driver.maxResultSize", "0") \
#     .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.3.3") \
#     .getOrCreate()

In [6]:
# !pip install -q --upgrade transformers[onnx]==4.35.2 optimum sentencepiece onnx==1.14.0
# MODEL_NAME = "flan-t5-small"
MODEL_NAME = "best_model"
EXPORT_PATH = f"onnx_models/{MODEL_NAME}"

In [7]:
!optimum-cli export onnx --task text2text-generation-with-past --model {MODEL_NAME} {EXPORT_PATH}

/root/anaconda3/envs/sp/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/anaconda3/envs/sp/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/root/anaconda3/envs/sp/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Framework not specified. Using pt to export the model.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, mak

In [8]:
!ls -l {EXPORT_PATH}

! mkdir -p {EXPORT_PATH}/assets
! mv -t {EXPORT_PATH}/assets {EXPORT_PATH}/spiece.model
!ls -l {EXPORT_PATH}/assets

total 810568
drwxr-xr-x 2 root root      4096 Jun  7 14:11 assets
-rw-r--r-- 1 root root      1517 Jun  7 14:34 config.json
-rw-r--r-- 1 root root 232553676 Jun  7 14:34 decoder_model.onnx
-rw-r--r-- 1 root root 232784389 Jun  7 14:35 decoder_model_merged.onnx
-rw-r--r-- 1 root root 219953983 Jun  7 14:35 decoder_with_past_model.onnx
-rw-r--r-- 1 root root 141456358 Jun  7 14:34 encoder_model.onnx
-rw-r--r-- 1 root root       142 Jun  7 14:34 generation_config.json
-rw-r--r-- 1 root root      2543 Jun  7 14:34 special_tokens_map.json
-rw-r--r-- 1 root root    791656 Jun  7 14:34 spiece.model
-rw-r--r-- 1 root root   2424156 Jun  7 14:34 tokenizer.json
-rw-r--r-- 1 root root     20817 Jun  7 14:34 tokenizer_config.json
total 776
-rw-r--r-- 1 root root 791656 Jun  7 14:34 spiece.model


In [9]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Installing PySpark 3.2.3 and Spark NLP 5.3.3
setup Colab for PySpark 3.2.3 and Spark NLP 5.3.3


In [10]:
import sparknlp

# let's start Spark with Spark NLP
spark = sparknlp.start()

In [11]:
from sparknlp.annotator import *

T5 = T5Transformer.loadSavedModel(EXPORT_PATH, spark)\
  .setUseCache(True) \
  .setTask("qa:") \
  .setMaxOutputLength(200)

Py4JJavaError: An error occurred while calling z:com.johnsnowlabs.nlp.annotators.seq2seq.T5Transformer.loadSavedModel.
: java.lang.NoClassDefFoundError: com/amazonaws/AmazonServiceException
	at com.johnsnowlabs.ml.util.LoadExternalModel$.modelSanityCheck(LoadExternalModel.scala:137)
	at com.johnsnowlabs.nlp.annotators.seq2seq.ReadT5TransformerDLModel.loadSavedModel(T5Transformer.scala:635)
	at com.johnsnowlabs.nlp.annotators.seq2seq.ReadT5TransformerDLModel.loadSavedModel$(T5Transformer.scala:633)
	at com.johnsnowlabs.nlp.annotators.seq2seq.T5Transformer$.loadSavedModel(T5Transformer.scala:694)
	at com.johnsnowlabs.nlp.annotators.seq2seq.T5Transformer.loadSavedModel(T5Transformer.scala)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflec

In [ ]:
T5.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

In [ ]:
!rm -rf {EXPORT_PATH}